<a href="https://colab.research.google.com/github/Mr-big-cake/Fundamental_AI/blob/main/Lab_3/AI_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import math
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import rc
from matplotlib import colors as cm
from typing import Tuple, Mapping
from enum import Enum
from numba import njit
import timeit

In [81]:
class Function(Enum):#Enum для выбора функций 
  MATIAS = 1
  SPHERE = 2
class SGD_Modification(Enum):#Enum для выбора модификаций
  SIMPLE = 1 # без модификаций
  MOMENT = 2 # моментная
  ADAPTIVE = 3 # Adam
class numbaGD():
  @staticmethod
  @njit
  def Matias (x: np.ndarray, y:np.ndarray) -> np.float128 :
    return  np.array(0.26*(x*x + y*y) - 0.48*x*y)
  @staticmethod
  @njit
  def Matias_dfdx(x:np.ndarray, y:np.ndarray)-> np.float128:
    return  np.array(0.52*x - 0.48*y)
  @staticmethod
  @njit
  def Matias_dfdy(x:np.ndarray, y:np.ndarray)-> np.float128:
    return np.array(-0.48*x + 0.52*y)
  @staticmethod
  @njit
  def Sphere (self,x: np.ndarray, y:np.ndarray,
              r: np.ndarray = 0.8)-> np.float128:
    return np.array(-np.sqrt(-x**2 - y**2 + r**2)) 
  @staticmethod
  @njit
  def Sphere_dfdx(self,x: np.ndarray, y:np.ndarray,
                  r: np.ndarray = 0.8)-> np.float128:
    return np.array(x/np.sqrt(r**2 - x**2 - y**2))
  @staticmethod
  @njit
  def Sphere_dfdy(self,x: np.ndarray, y:np.ndarray,
                  r: np.ndarray = 0.8)-> np.float128:
    return np.array(np.array(y/np.sqrt(r**2 - x**2 - y**2)))

  @staticmethod
  def GD_Plot_CPU(mod:SGD_Modification, f: Mapping, fdfdx: Mapping, fdfdy:Mapping, xt: np.ndarray, yt: np.ndarray, lr: float = 0.01, count_epochs: int = 10000):
    for c_e in np.arange(count_epochs):
      dfdx = fdfdx(xt, yt)
      dfdy = fdfdy(xt, yt)
      if (mod == SGD_Modification.SIMPLE):
        xt = xt - lr * dfdx
        yt = yt - lr * dfdy
      elif(mod == SGD_Modification.MOMENT):
        kb = 0.9
        xt = kb * xt - (1 - kb) * dfdx
        yt = kb * yt - (1 - kb) * dfdy
      elif(mod == SGD_Modification.ADAPTIVE):
        k1, k2, eps, v_x, v_y, G_x, G_y = 0.9, 0.99, 1e-8, 0, 0, 0, 0
        v_x = k1 * v_x + (1 - k1) * dfdx
        v_y = k1 * v_y + (1 - k1) * dfdy
        G_x = k2 * G_x + (1 - k2) * (dfdx) ** 2
        G_y = k2 * G_y + (1 - k2) * (dfdy) ** 2
        xt = xt - lr * v_x / np.sqrt(G_x + eps) ** (1 / 2)
        yt = yt - lr * v_y / np.sqrt(G_y + eps) ** (1 / 2)
        
    return xt, yt, f(xt, yt)

  @staticmethod
  @njit
  def GD_Plot(mod:SGD_Modification, f: Mapping, fdfdx: Mapping, fdfdy:Mapping, xt: np.ndarray, yt: np.ndarray, lr: float = 0.01, count_epochs: int = 10000):
    for c_e in np.arange(count_epochs):
      dfdx = fdfdx(xt, yt)
      dfdy = fdfdy(xt, yt)
      if (mod == SGD_Modification.SIMPLE):
        xt = xt - lr * dfdx
        yt = yt - lr * dfdy
      elif(mod == SGD_Modification.MOMENT):
        kb = 0.9
        xt = kb * xt - (1 - kb) * dfdx
        yt = kb * yt - (1 - kb) * dfdy
      elif(mod == SGD_Modification.ADAPTIVE):
        k1, k2, eps, v_x, v_y, G_x, G_y = 0.9, 0.99, 1e-8, 0, 0, 0, 0
        v_x = k1 * v_x + (1 - k1) * dfdx
        v_y = k1 * v_y + (1 - k1) * dfdy
        G_x = k2 * G_x + (1 - k2) * (dfdx) ** 2
        G_y = k2 * G_y + (1 - k2) * (dfdy) ** 2
        xt = xt - lr * v_x / np.sqrt(G_x + eps) ** (1 / 2)
        yt = yt - lr * v_y / np.sqrt(G_y + eps) ** (1 / 2)
        
    return xt, yt, f(xt, yt)

    @staticmethod
    @njit
    def Plot_Plot(f, p_x, p_y, p_z, history, h= 80 , a = 55,
                difference = 1,  rad = 0.8, opt = 0.3):
      log = history
      fig = plt.figure(figsize=(20,15))
      x = np.linspace(-difference, difference, 100) 
      y = np.linspace(-difference, difference, 100) 
      X, Y = np.meshgrid(x, y) 
      Z = f(X, Y) 
      ax = plt.axes(projection ='3d') 
      ax.view_init(h, a) # Высота и угол  
      ax.plot_wireframe(X, Y, Z, color ='blue', alpha = 0.1) 
      ax.scatter(log[0][0], log[0][1], log[0][2], linewidths  = 5 ,
               color='yellow', alpha = 1, label = 'Начальная точка')
      ax.scatter(p_x, p_y, p_z, linewidths  = 5 ,
               color='red', alpha = 1, label = 'Результат работы SGD')
      ax.scatter(0, 0, -rad, linewidths  = 8 ,
               color='black', alpha = 1, label = 'Оптимум')
      ax.legend(fontsize=10)
      ax.plot3D(np.linspace(p_x, p_x, 2), np.linspace(p_y, p_y, 2),
              np.linspace(p_z, opt, 2), 'red')
      dlog = 1 
      for i in range(0, len(log) - dlog, dlog):
        ax.plot3D(np.linspace(log[i][0], log[i+1][0] , 2),
                np.linspace(log[i][1], log[i+1][1], 2), 
                np.linspace(log[i][2], log[i+1][2], 2), 'grey', alpha = 0.4)

In [78]:
x, y, z  = numbaGD.GD_Plot(SGD_Modification.SIMPLE, numbaGD.Matias, numbaGD.Matias_dfdx, numbaGD.Matias_dfdy, 0.5, 0.5 )
print("Результат работы numbaGD")
print("x = ", x,"\ny = ", y, "\nf(x, y) = ", z)

Результат работы numbaGD
x =  0.009150494165841029 
y =  0.009150494165841029 
f(x, y) =  3.3492617391636185e-06


In [79]:
x, y, z  = numbaGD.GD_Plot(SGD_Modification.MOMENT, numbaGD.Matias, numbaGD.Matias_dfdx, numbaGD.Matias_dfdy, 0.5, 0.5 )
print("Результат работы numbaGD с мометной модификацией")
print("x = ", x,"\ny= ", y, "\nf(x, y) = ", z)

Результат работы numbaGD с мометной модификацией
x =  2.5e-323 
y=  2.5e-323 
f(x, y) =  0.0


In [80]:
x, y, z  = numbaGD.GD_Plot(SGD_Modification.ADAPTIVE, numbaGD.Matias, numbaGD.Matias_dfdx, numbaGD.Matias_dfdy, 0.5, 0.5 )
print("Результат работы numbaGD с адаптивной модификацией")
print("x = ", x,"\ny= ", y, "\nf(x, y) = ", z)

Результат работы numbaGD с адаптивной модификацией
x =  1.323702117784267e-16 
y=  1.323702117784267e-16 
f(x, y) =  7.008749186506224e-34


In [ ]:
print("Время работы без numba:")
print(timeit.timeit("numbaGD.GD_Plot_CPU(SGD_Modification.MOMENT, numbaGD.Matias, numbaGD.Matias_dfdx, numbaGD.Matias_dfdy, 0.5, 0.5 )", globals=globals()))

Время работы без numba:


In [ ]:
print("Время работы c numba:")
print(timeit.timeit("numbaGD.GD_Plot(SGD_Modification.MOMENT, numbaGD.Matias, numbaGD.Matias_dfdx, numbaGD.Matias_dfdy, 0.5, 0.5 )", globals=globals()))